Add miniLM score for (generated_answer, extracted_answer).

In [28]:
import json
from sentence_transformers import CrossEncoder

In [29]:
def read_rows(path):
    rows = []
    for line in open(path):
        rows.append(json.loads(line))
    return rows

def write_json_format(path_out, rows):
    f_out = open(path_out, 'w')
    for row in rows:
        f_out.write(json.dumps(row, ensure_ascii=False)+'\n')

In [30]:
path_dataset = 'test-B-big/dev.jl'
path_model = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

In [31]:
rows = read_rows(path_dataset)

In [32]:
inputs_gpt3 = [(row['gpt3_answer'], row['distillbert_answer']) for row in rows]
inputs_chatgpt = [(row['chatgpt_answer'], row['distillbert_answer']) for row in rows]

In [33]:
model = CrossEncoder(path_model, max_length=512)

In [34]:
scores_gpt3 = model.predict(inputs_gpt3)
scores_chatgpt = model.predict(inputs_chatgpt)

In [35]:
for row, score_gpt3, score_chatgpt in zip(rows, scores_gpt3, scores_chatgpt):
    row['score_miniLM_gpt3_answer'] = float(score_gpt3)
    row['score_miniLM_chatgpt_answer'] = float(score_chatgpt)

In [36]:
write_json_format(path_dataset, rows)